# Notes

You might want to check https://julkaar9.github.io/walkalongs/orms/django/w3school/ to get  
basic an idea on how to solve these questions.  

Make sure to check the column names and foreign key relationships from server/customer_db/hospital_models.py.

Use `sql_to_df` to print the sql query results.  

Some insights on the equal function. 
1. Proper type casts might be required to get an equal result
2. Exact ordering of columns is not required for query results to be equal
3. Exact ordering of rows / tuples however are required.

In [1]:
import os

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from utils import *
from sqlparse import format

In [2]:
from django.db.models import (
    Q,
    F,
    Case,
    When,
    Count,
    Func,
    Min,
    Max,
    Sum,
    Avg,
    Value,
    OuterRef,
    Subquery,
    CharField,
)
from django.db.models.functions import Concat, Cast, Round, Length, Upper, Lower
from django.db import connection
from customer_db.models import Provinces, Patients, Doctors, Admissions

In [3]:
def print_sql(queryset):
    formatted = format(str(queryset.query), reindent=True)
    print(formatted)

In [4]:
admission_fields = [
    "id",
    "patient_id",
    "admission_date",
    "discharge_date",
    "diagnosis",
    "attending_doctor_id",
]

In [5]:
class NonAggrCount(Count):
    """
    This aggregation will not trigger a group by
    """

    contains_aggregate = False


class NonAggrSum(Sum):
    contains_aggregate = False

# Example 
Show first_name of patient and admission date from admission table. Order by first_name and admission_date

In [6]:
qstr = """
SELECT
  p.first_name,
  admission_date
FROM admissions a
JOIN patients p on p.patient_id = a.patient_id
ORDER By p.first_name, admission_date
"""
sqlq = sql_raw(qstr)

In [7]:
ormq = Admissions.objects.values(
    "admission_date", first_name=F("patient__first_name")
).order_by("first_name", "admission_date")
equal(sqlq, ormq)
print_sql(ormq)

Equal ✔️
SELECT "admissions"."admission_date",
       "patients"."first_name" AS "first_name"
FROM "admissions"
INNER JOIN "patients" ON ("admissions"."patient_id" = "patients"."patient_id")
ORDER BY 2 ASC,
         "admissions"."admission_date" ASC


In [8]:
orm_to_df(ormq[:3])

,admission_date,first_name
0,2022-02-18,Abigail
1,2022-05-28,Abigail
2,2022-01-08,Adam


## Medium 1
Show unique birth years from patients and order them by ascending.

In [9]:
# Use YEAR(birth_date) for most databases
qstr = """
SELECT
  DISTINCT strftime('%Y', birth_date) birth_year
FROM patients
ORDER BY birth_year;
"""
sqlq = sql_raw(qstr)

In [10]:
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [11]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Medium 2
Show unique first names from the patients table which only occurs once in the list.

For example, if two or more people are named 'John' in the first_name column then don't include their name in the output list. If only 1 person is named 'Leo' then include them in the output.

In [12]:
qstr = """
SELECT first_name
FROM patients
GROUP BY first_name
HAVING COUNT(first_name) = 1
"""
sqlq = sql_raw(qstr)

In [13]:
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [14]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Medium 3
Show patient_id and first_name from patients where their first_name start and ends with 's' and is at least 6 characters long.
We will do start and ends with 'n' and is at least 6 characters long.

In [15]:
qstr = """
SELECT
  patient_id,
  first_name
FROM patients
WHERE first_name LIKe "n____%n"
"""
sqlq = sql_raw(qstr)

In [16]:
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [17]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Medium 4
Show patient_id, first_name, last_name from patients whos diagnosis is 'Dementia'.
We will do Diabetes.
Primary diagnosis is stored in the admissions table.

In [18]:
qstr = """
SELECT
  p.patient_id,
  first_name,
  last_name
FROM patients p
  INNER JOIN admissions a ON p.patient_id = a.patient_id
WHERE diagnosis = 'Diabetes'
"""
sqlq = sql_raw(qstr)

In [19]:
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [20]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Medium 5
Display every patient's first_name.
Order the list by the length of each name and then by alphbetically

In [21]:
qstr = """
SELECT
  first_name
FROM patients p
ORDER BY
  LENGTH(first_name),
  first_name;
"""
sqlq = sql_raw(qstr)

In [22]:
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [23]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Medium 6
Show the total amount of male patients and the total amount of female patients in the patients table.
Display the two results in the same row.

In [24]:
qstr = """
SELECT (
    SELECT COUNT(gender)
    FROM patients
    WHERE gender = 'M'
  ) AS male, (
    SELECT COUNT(gender)
    FROM patients
    WHERE gender = 'F'
  ) AS female;
"""
sqlq = sql_raw(qstr)

In [25]:
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [26]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Medium 7
Show first and last name, allergies from patients which have allergies to either 'Penicillin' or 'Morphine'. Show results ordered ascending by allergies then by first_name then by last_name.

In [27]:
qstr = """
SELECT
  first_name,
  last_name,
  allergies
FROM patients
WHERE
  allergies IN ('Penicillin', 'Morphine')
ORDER BY
  allergies,
  first_name,
  last_name;
"""
sqlq = sql_raw(qstr)

In [28]:
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [29]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Medium 8
Show patient_id, diagnosis from admissions. Find patients admitted multiple times for the same diagnosis.

In [30]:
qstr = """
SELECT
  patient_id,
  diagnosis
FROM admissions
GROUP BY
  patient_id,
  diagnosis
HAVING COUNT(*) > 1
"""
sqlq = sql_raw(qstr)

In [31]:
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [32]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Medium 9
Show the city and the total number of patients in the city.
Order from most to least patients and then by city name ascending.

In [33]:
qstr = """
SELECT
  city,
  COUNT(*) population
FROM patients
GROUP BY city 
ORDER BY population DESC, city
"""
sqlq = sql_raw(qstr)

In [34]:
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [35]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Medium 10
Show first name, last name and role of every person that is either patient or doctor.
The roles are either "Patient" or "Doctor"

In [36]:
qstr = """
SELECT
  first_name,
  last_name,
  'patient' AS Role
FROM patients
UNION ALL
SELECT
  first_name,
  last_name,
  'doctor' AS Role
FROM doctors;
"""
sqlq = sql_raw(qstr)

In [37]:
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [38]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Medium 11
Show all allergies and their occurance ordered by occurance. Remove NULL values from query.

In [39]:
qstr = """
SELECT
  allergies,
  COUNT (*) AS total_occurance
FROM patients
WHERE allergies IS NOT null
GROUP BY allergies
ORDER BY total_occurance DESC
"""
sqlq = sql_raw(qstr)

In [40]:
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [41]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Medium 12
Show all patient's first_name, last_name, and birth_date who were born in the 1970s decade. Sort the list starting from the earliest birth_date.

In [42]:
# use YEAR(birth_date) if year function is available
qstr = """
SELECT
  first_name,
  last_name,
  birth_date
FROM patients
WHERE CAST(strftime('%Y', birth_date) AS INT) BETWEEN 1970 AND 1979
ORDER BY birth_date
"""
sqlq = sql_raw(qstr)

In [43]:
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [44]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Medium 13
We want to display each patient's full name in a single column. Their last_name in all upper letters must appear first, then first_name in all lower case letters. Separate the last_name and first_name with a comma. Order the list by the first_name in decending order
EX: SMITH,jane

In [45]:
qstr = """
SELECT
   UPPER(last_name) || ',' || LOWER(first_name) AS full_name
FROM patients
ORDER BY first_name DESC
"""
sqlq = sql_raw(qstr)

In [46]:
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [47]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Medium 14
Show the province_id(s), sum of height; where the total sum of its patient's height is greater than or equal to 7,000.
We will do 3000

In [48]:
qstr = """
SELECT
  province_id,
  SUM(height) AS sum_height
FROM patients
GROUP BY province_id
HAVING SUM(height) > 3000;
"""
sqlq = sql_raw(qstr)

In [49]:
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [50]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Medium 15
Show the difference between the largest weight and smallest weight for patients with the last name 'Maroni'.
We will do Lee.

In [51]:
qstr = """
SELECT
  (MAX(weight) - MIN(weight)) AS weight_dif
FROM patients
WHERE last_name = 'Lee';
"""
sqlq = sql_raw(qstr)

In [52]:
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
# print_sql(ormq)

Unequal ❌


In [53]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Medium 16
Show all of the days of the month (1-31) and how many admission_dates occurred on that day. Sort by the day with most admissions to least admissions.

In [54]:
qstr = """
SELECT
  CAST(strftime('%d', admission_date) as INT) AS month_day,
  COUNT(patient_id) AS daily_admission
FROM admissions
GROUP BY strftime('%d', admission_date)
ORDER BY daily_admission DESC;
"""
sqlq = sql_raw(qstr)

In [55]:
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [56]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Medium 17
Show all columns for patient_id 54's most recent admission_date.

In [57]:
qstr = """
SELECT *
FROM admissions
WHERE patient_id = 54 AND admission_date = (
    SELECT max(admission_date)
    FROM admissions
    WHERE patient_id = 54
  )
"""
sqlq = sql_raw(qstr)

In [58]:
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [59]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Medium 18
Show patient_id, attending_doctor_id, and diagnosis for admissions that match one of the two criteria:
1. patient_id is an odd number and attending_doctor_id is either 1, 5, or 19.
2. attending_doctor_id contains a 2 and the length of patient_id is 3 characters.

In [60]:
qstr = """
SELECT
  patient_id,
  attending_doctor_id,
  diagnosis
FROM admissions
WHERE
  (
    patient_id % 2 = 1
    and attending_doctor_id IN (1, 5, 19)
  )
  OR (
    CAST(attending_doctor_id AS varchar) LIKE "%2%"
    AND length(CAST(patient_id AS VARCHAR)) = 2
  )
"""
sqlq = sql_raw(qstr)

In [61]:
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [62]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Medium 19
Show first_name, last_name, and the total number of admissions attended for each doctor.

Every admission has been attended by a doctor.

In [63]:
qstr = """
SELECT
  first_name,
  last_name,
  COUNT(*) AS patient_count
FROM admissions
  JOIN doctors ON attending_doctor_id = doctor_id
GROUP BY attending_doctor_id
"""
sqlq = sql_raw(qstr)

In [64]:
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [65]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Medium 20
For each doctor, display their id, full name, and the first and last admission date they attended.

In [66]:
qstr = """
SELECT
  doctor_id,
  first_name || ' ' || last_name AS full_name,
  MIN(admission_date) AS min_admission,
  MAX(admission_date) AS max_admission
FROM doctors
  JOIN admissions ON attending_doctor_id = doctor_id
GROUP BY doctor_id
"""
sqlq = sql_raw(qstr)

In [67]:
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [68]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Medium 21
Display the total amount of patients for each province. Order by descending.

In [69]:
qstr = """
SELECT
  pn.province_name,
  count(patient_id) population
FROM patients p
  JOIN province_names pn ON p.province_id = pn.province_id
GROUP BY pn.province_id
ORDER BY population desc
"""
sqlq = sql_raw(qstr)

In [70]:
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [71]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Medium 22
For every admission, display the patient's full name, their admission diagnosis, and their doctor's full name who diagnosed their problem.

In [72]:
qstr = """
SELECT
  p.first_name || ' ' || p.last_name AS patient_name,
  diagnosis,
  d.first_name || ' ' || d.last_name AS doctor_name
FROM patients p
  JOIN admissions a ON p.patient_id = a.patient_id
  JOIN doctors d ON a.attending_doctor_id = d.doctor_id
"""
sqlq = sql_raw(qstr)

In [73]:
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [74]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Medium 23
display the number of duplicate patients based on their first_name and last_name.

In [75]:
qstr = """
SELECT
  first_name,
  last_name,
  count(*) duplicates
FROM patients
GROUP BY
  first_name,
  last_name
HAVING COUNT(*) > 1
"""
sqlq = sql_raw(qstr)

In [76]:
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [77]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Medium 24
Display patient's full name,
height in the units feet rounded to 1 decimal,
weight in the unit pounds rounded to 0 decimals,
birth_date,
gender non abbreviated.

Convert CM to feet by dividing by 30.48.
Convert KG to pounds by multiplying by 2.205.

In [78]:
qstr = """
SELECT
  first_name || ' ' || last_name AS full_name,
  ROUND(height / 30.48, 1) AS height_inc,
  ROUND(weight * 2.205, 0) AS weight_pound,
  birth_date,
  CASE
    WHEN gender = 'M' THEN 'Male'
    WHEN gender = 'F' THEN 'Female'
  END AS gender_full
FROM patients
"""
sqlq = sql_raw(qstr)

In [79]:
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [80]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Medium 25
Show patient_id, first_name, last_name from patients whose does not have any records in the admissions table. (Their patient_id does not exist in any admissions.patient_id rows.)

In [81]:
qstr = """
SELECT
  patient_id,
  first_name,
  last_name
FROM patients
WHERE patient_id NOT IN (
    SELECT patient_id
    FROM admissions
  )
"""
sqlq = sql_raw(qstr)

In [82]:
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [83]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3
